In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="XXXXXXXXXXX", parse_args=False, project_name='XXXXXXXX')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.54 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/5fb9fb2bbf58464b8853d5ef18571abe



In [1]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtPrintNoIntsectHVBW,ArtPrintNoIntsectHVBW_biameyd_siameyd,ArtPrintNoIntsectHVBW_bpr_spr,ArtPrintNoIntsectHVBW_biameyd_sprt
from Model_Unet_github import *


home = os.environ['HOME']

ImportError: No module named 'data_artifact_edit_w_spurious_words'

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='segnet_unet_hvbw_all_combine_100_epoches', type=str, help="name of the log") #debug model_intersect # segnet_no_intersect_1conv_64_channels_30epoch_unet_github
parser.add_argument("-gpu", default='0', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=60, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/root/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/root/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name(name)
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arhh=ArtPrintNoIntsectHVBW_biameyd_siameyd(args.data_root, transform=transform_train)
        arph=ArtPrintNoIntsectHVBW(args.data_root, transform=transform_train)
        arpp=ArtPrintNoIntsectHVBW_bpr_spr(args.data_root, transform=transform_train)
        arhp=ArtPrintNoIntsectHVBW_biameyd_sprt(args.data_root, transform=transform_train)
        concat=ConcatDataset([arhh,arph,arpp,arhp])
        print(len(concat))
        idxTrain = int( len(concat)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        #return trainset, concat
        #weight gen
        # modified here:
        samples_all=[]
        print(trainset.dataset.datasets)
        for ds in trainset.dataset.datasets:
            samples_all+=ds.samples
        #samples_all=trainset.dataset.samples
        print(len(samples_all))
        print(samples_all[0])
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),samples_all))/(args.image_h*args.image_w*len(samples_all))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [ ]:
main()

The model is set to Training
Max training Iteration: 60
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /root/datasets
Check point file: /root/ckpt/segnet_unet_hvbw_all_combine_100_epoches
Batch Size: 10
822980
[<datasets.ArtPrintNoIntsectHVBW_biameyd_siameyd object at 0x7ff4f0e76470>, <datasets.ArtPrintNoIntsectHVBW object at 0x7ff4f0e76358>, <datasets.ArtPrintNoIntsectHVBW_bpr_spr object at 0x7ff4f0e76748>, <datasets.ArtPrintNoIntsectHVBW_biameyd_sprt object at 0x7ff4e6d8b588>]
822980
('/root/datasets/artifact_images_noins_hvbw_biameyd_siameyd/images/d06-067___d06-067-09-00.png', '/root/datasets/artifact_images_noins_hvbw_biameyd_siameyd/labels/d06-067___d06-067-09-00.png', 'd06-067-09-00')
[0.0881255 0.9118745]
Instructions for updating:
Colocations handled automatically by placer.
GGG
[None, 32, 128, 1]
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


0 conv1: (?, ?, ?, 32)
0 conv2: (?, ?, ?, 32)
1 conv1: (?, ?, ?, 64)
1 conv2: (?, ?, ?, 64)
2 conv1: (?, ?, ?, 128)
2 conv2: (?, ?, ?, 128)
1 h_deconv: (?, ?, ?, 64)
1 h_deconv_concat: (?, ?, ?, ?)
1 h_conv1_post_deconv: (?, ?, ?, 64)
1 h_conv2_post_deconv: (?, ?, ?, 64)
0 h_deconv: (?, ?, ?, 32)
0 h_deconv_concat: (?, ?, ?, ?)
0 h_conv1_post_deconv: (?, ?, ?, 32)
0 h_conv2_post_deconv: (?, ?, ?, 32)
0 outmap: (?, ?, ?, 2)
(?, ?, ?, 2)
loss: ()
INFO:tensorflow:Summary name loss/cross_entropy (raw) is illegal; using loss/cross_entropy__raw_ instead.


...

TRAIN: Batch: 0.0 Loss: 0.010558588
accuracy = 0.973331
mean IU  = 0.869207
    class # 0 capture rate = 0.972603 
    class # 1 capture rate = 0.980701 
TRAIN: Batch: 0.0013501107090781444 Loss: 0.011301552
accuracy = 0.973040
mean IU  = 0.869064
    class # 0 capture rate = 0.972096 
    class # 1 capture rate = 0.982499 
TRAIN: Batch: 0.0027002214181562887 Loss: 0.016443502
accuracy = 0.951868
mean IU  = 0.799731
    class # 0 capture rate = 0.948949 
    class # 1 capture rate = 0.980637 
TRAIN: Batch: 0.004050332127234433 Loss: 0.012586984
accuracy = 0.966856
mean IU  = 0.843250
    class # 0 capture rate = 0.965502 
    class # 1 capture rate = 0.980871 
TRAIN: Batch: 0.0054004428363125774 Loss: 0.015045574
accuracy = 0.958819
mean IU  = 0.809326
    class # 0 capture rate = 0.957598 
    class # 1 capture rate = 0.972214 
TRAIN: Batch: 0.0067505535453907224 Loss: 0.009761622
accuracy = 0.970990
mean IU  = 0.863573
    class # 0 capture rate = 0.968983 
    class # 1 capture rate

TRAIN: Batch: 0.06615542474482908 Loss: 0.012294209
accuracy = 0.959625
mean IU  = 0.830326
    class # 0 capture rate = 0.956572 
    class # 1 capture rate = 0.987768 
TRAIN: Batch: 0.06750553545390722 Loss: 0.0118312035
accuracy = 0.969035
mean IU  = 0.847640
    class # 0 capture rate = 0.967844 
    class # 1 capture rate = 0.981871 
TRAIN: Batch: 0.06885564616298537 Loss: 0.007867088
accuracy = 0.976876
mean IU  = 0.854422
    class # 0 capture rate = 0.976144 
    class # 1 capture rate = 0.987500 
TRAIN: Batch: 0.07020575687206351 Loss: 0.011545645
accuracy = 0.968671
mean IU  = 0.846220
    class # 0 capture rate = 0.967139 
    class # 1 capture rate = 0.985264 
TRAIN: Batch: 0.07155586758114166 Loss: 0.010925624
accuracy = 0.970966
mean IU  = 0.850602
    class # 0 capture rate = 0.970050 
    class # 1 capture rate = 0.981337 
TRAIN: Batch: 0.0729059782902198 Loss: 0.009508354
accuracy = 0.974502
mean IU  = 0.880786
    class # 0 capture rate = 0.972695 
    class # 1 captu

TRAIN: Batch: 0.13231084948965816 Loss: 0.012768534
accuracy = 0.961784
mean IU  = 0.828351
    class # 0 capture rate = 0.959405 
    class # 1 capture rate = 0.985870 
TRAIN: Batch: 0.1336609601987363 Loss: 0.01624655
accuracy = 0.950583
mean IU  = 0.773759
    class # 0 capture rate = 0.948864 
    class # 1 capture rate = 0.971301 
TRAIN: Batch: 0.13501107090781445 Loss: 0.013313884
accuracy = 0.957318
mean IU  = 0.809845
    class # 0 capture rate = 0.955483 
    class # 1 capture rate = 0.976503 
TRAIN: Batch: 0.1363611816168926 Loss: 0.009507971
accuracy = 0.970496
mean IU  = 0.846450
    class # 0 capture rate = 0.969395 
    class # 1 capture rate = 0.983318 
TRAIN: Batch: 0.13771129232597074 Loss: 0.009947621
accuracy = 0.970842
mean IU  = 0.867407
    class # 0 capture rate = 0.968922 
    class # 1 capture rate = 0.988685 
TRAIN: Batch: 0.13906140303504888 Loss: 0.011843814
accuracy = 0.963222
mean IU  = 0.842397
    class # 0 capture rate = 0.960820 
    class # 1 capture 

TRAIN: Batch: 0.19846627423448723 Loss: 0.011588295
accuracy = 0.965460
mean IU  = 0.809947
    class # 0 capture rate = 0.964781 
    class # 1 capture rate = 0.974793 
TRAIN: Batch: 0.19981638494356538 Loss: 0.014916693
accuracy = 0.961186
mean IU  = 0.862828
    class # 0 capture rate = 0.958150 
    class # 1 capture rate = 0.981035 
TRAIN: Batch: 0.20116649565264352 Loss: 0.008180288
accuracy = 0.973460
mean IU  = 0.852408
    class # 0 capture rate = 0.972342 
    class # 1 capture rate = 0.987455 
TRAIN: Batch: 0.20251660636172167 Loss: 0.011097473
accuracy = 0.969690
mean IU  = 0.863873
    class # 0 capture rate = 0.968069 
    class # 1 capture rate = 0.984577 
TRAIN: Batch: 0.2038667170707998 Loss: 0.0153217185
accuracy = 0.954504
mean IU  = 0.816935
    class # 0 capture rate = 0.951485 
    class # 1 capture rate = 0.981531 
TRAIN: Batch: 0.20521682777987796 Loss: 0.009397947
accuracy = 0.972011
mean IU  = 0.860576
    class # 0 capture rate = 0.970610 
    class # 1 captu

TRAIN: Batch: 0.2646216989793163 Loss: 0.013990967
accuracy = 0.961587
mean IU  = 0.827305
    class # 0 capture rate = 0.959771 
    class # 1 capture rate = 0.979935 
TRAIN: Batch: 0.26597180968839446 Loss: 0.011469674
accuracy = 0.967537
mean IU  = 0.825071
    class # 0 capture rate = 0.967078 
    class # 1 capture rate = 0.973401 
TRAIN: Batch: 0.2673219203974726 Loss: 0.016550008
accuracy = 0.952522
mean IU  = 0.778677
    class # 0 capture rate = 0.951145 
    class # 1 capture rate = 0.969221 
TRAIN: Batch: 0.26867203110655075 Loss: 0.011749648
accuracy = 0.964235
mean IU  = 0.834856
    class # 0 capture rate = 0.962162 
    class # 1 capture rate = 0.985623 
TRAIN: Batch: 0.2700221418156289 Loss: 0.009875911
accuracy = 0.970325
mean IU  = 0.862175
    class # 0 capture rate = 0.968997 
    class # 1 capture rate = 0.983121 
TRAIN: Batch: 0.27137225252470704 Loss: 0.011274171
accuracy = 0.966446
mean IU  = 0.831507
    class # 0 capture rate = 0.964729 
    class # 1 capture 